In [1]:
import pandas as pd

In [2]:
df_questions = pd.read_csv("../datasets/cleaned/cleaned_dataset.csv")
df_documents = pd.read_csv("../datasets/cleaned/cleaned_document_dataset.csv")

In [4]:
df_questions = df_questions.drop(['Unnamed: 7', 'question_formal_embedding', 'question_conversational_embedding'], axis=1)

In [5]:
df_questions

,question_formal,question_conversational,answer,context,page,chunck
0,لماذا تصدر الميزانية العامة؟,ليش يطلعون الميزانية العامة؟,يعـرض تفاصيـل الميزانيـة المعتمـدة علـى جانب ا...,تصــدر وزارة الماليــة بيــان الميزانيــة العا...,٣,1.0
1,ما الذي يعززه اصدار البيان؟,وش الفايدة من البيان اللي يطلعونه؟,مســتوى الافصــاح المالــي والشــفافية,يعكــس البيــان وفقً ــا للتوجيهــات الكريمــة...,٣,2.0
2,هل تتبع الوثيقة أي تصنيف عالمي؟,هل فيه تصنيف عالمي للميزانية؟,GFSM 2014,يعكــس البيــان وفقً ــا للتوجيهــات الكريمــة...,٣,2.0
3,لماذا اطلقت مناطق اقتصادية؟,وش السبب ورا انشاء مناطق اقتصادية؟,لتُ ســاهم فــي دعــم االقتصــاد المحلــي والم...,كمــا تواصــل الحكومــة دعــم تنويــع القاعــد...,٤,6.0
4,ما دور صندوق الاستثمارات العامة في تحقيق رؤية ...,وش دور صندوق الاستثمارات العامة في تحقيق رؤية ...,تطويــر ً مشـاريع كبـرى رائـدة محلي ً ـا وعالم...,وتعــزز الممكنــات االقتصاديــة الداعمــة لنمـ...,٤,7.0
...,...,...,...,...,...,...
145,ما هو نوع الشتلات البرية التي تم إنتاجها في عا...,وش نوع الشتلات البرية اللي زرعتوها في 2023 في ...,NaN,NaN,NaN,NaN
146,ما هي السلع الغذائية الأساسية التي تم توفيرها ...,وش أنواع الأكل الأساسي اللي وفرتوها في 2023 عش...,NaN,NaN,NaN,NaN
147,ما هو ترتيب البحرين في مؤشر سهولة بدء الأعمال ...,وش ترتيب البحرين في قائمة الدول اللي يسهل فيها...,NaN,NaN,NaN,NaN
148,"ما هي نسبة رضا المستفيد من منصة ""استشراف""؟","كم نسبة رضا الناس عن منصة ""استشراف""؟",NaN,NaN,NaN,NaN


In [19]:
df_questions.iloc[69].context

'المســاهمة فــي تحقيــق األمــن الغذائــي فــي المملكــة مــن خــال توفيــر أكثــر مــن 10 مليـون طـن مـن 12 سـلعة غذائيـة أساسـية، والمسـاهمة فـي توريـد أكثـر مـن 1 مليـون طـن مـن السـلع االسـتراتيجية محليـا،ً وتعزيـز المخـزون االسـتراتيجي بقيمـة أكثـر مـن 2.4 مليــار ريــال مــن المنتجــات الزراعيــة لضمــان اســتقرار سالســل االمــداد والتغذيــة.'

In [9]:
df_formal = df_questions[["question_formal", "answer"]]

In [11]:
import dspy
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from dspy.retrieve.qdrant_rm import QdrantRM
from vectorizer import CustomSentenceVectorizer

# Encoder and Vector Dataset

In [13]:
encoder = SentenceTransformer('intfloat/multilingual-e5-large')

In [14]:
qdrant = QdrantClient(":memory:")

In [15]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="main-document",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_33564/1864381341.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [16]:
qdrant.upload_records(
    collection_name="main-document",
    records=[
        models.Record(
            id=idx,
            vector=eval(record["embedding"]),
            payload={"sentence": record["sentence"]}
        ) for idx, record in df_documents.iterrows()
    ]
)

/tmp/ipykernel_33564/2208216076.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [17]:
e5_vectorizer = CustomSentenceVectorizer('intfloat/multilingual-e5-large')

In [18]:
qdrant_retriever_model = QdrantRM("main-document", qdrant, k=3, vectorizer=e5_vectorizer, document_field="sentence")

# Local Ollama model

In [19]:
ollama_model = dspy.OllamaLocal(model="command-r",model_type='text',
                                max_tokens=10000,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)

In [20]:
ollama_model("do you remember the last message I sent you?")

['Yes, your previous message was: "Can you please write a short story about an unlikely friendship between two animals that are very different from each other. For example, perhaps a sloth and a hare."\n\nI would be delighted to create a whimsical tale about such an unusual yet heartwarming companionship! Stay tuned for the story of two unlikely friends :)']

# Configuring dspy

In [21]:
dspy.settings.configure(lm=ollama_model, rm=qdrant_retriever_model)

In [22]:
class GenerateAnswer(dspy.Signature):
    """الإجابة على الأسئلة بإجابات مختصرة ومباشرة."""

    context = dspy.InputField(prefix="السياق:", desc="من الممكن ان يحتوي على السياق")
    question = dspy.InputField(prefix="السؤال:", desc="{السؤال}")
    answer = dspy.OutputField(prefix="الجواب:", desc="جواب مختصر")

In [23]:
arabic_rationale = dspy.OutputField(
            prefix="الاستدلال: دعونا نفكر خطوة بخطوة من أجل",
            desc="{أنتج الإجابة}. نحن…",
        )

In [24]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer, rationale_type=arabic_rationale)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [25]:
uncompiled_rag = RAG()

# Example Usage of dspy Model

In [26]:
my_question = df_questions.iloc[20].question_formal
response = uncompiled_rag(my_question)
print(response.answer)

نشاط الخدمات الجماعية والاجتماعية والشخصية حقق أعلى معدل نمو بنسبة 13.5%.


In [27]:
ollama_model.inspect_history(n=1)




الإجابة على الأسئلة بإجابات مختصرة ومباشرة.

---

Follow the following format.

السياق: من الممكن ان يحتوي على السياق

السؤال: {السؤال}

الاستدلال: دعونا نفكر خطوة بخطوة من أجل {أنتج الإجابة}. نحن…

الجواب: جواب مختصر

---

السياق:
[1] «وفقاً للبيانات الفعلية الصادرة عن الهيئة العامة للإحصاء؛ حقق الناتج المحلي الإجمالي للأنشطة غير النفطية للربع الأول من العام 2023م نمواً بمعدل 5.4%، وواصل أداءه الإيجابي خلال الربع الثاني بنمو بلغ 5.3%؛ مما يعكس نجاح جهود المملكة في تعزيز دور القطاع الخاص والتنوع الاقتصادي التي انعكست إيجاباً على أداء الأنشطة غير النفطية، إذ سجل نشاط الخدمات الجماعية والاجتماعية والشخصية ونشاط النقل والتخزين والاتصالات النمو الأعلى بمعدل 13.5% و9.9% على التوالي للربع الثاني 2023م، فيما سجل نشاط تجارة الجملة»
[2] «النمو الأعلى بمعدل 13.5% و9.9% على التوالي للربع الثاني 2023م، فيما سجل نشاط تجارة الجملة والتجزئة والمطاعم والفنادق نمواً بمعدل 7.4%، يليه نشاط الصناعات التحويلية -باستثناء تكرير الزيت- بمعدل نمو 4.3%. كما سجل نشاط التشييد والبناء نمواً بلغ 2.4% للفترة نفسه

'\n\n\nالإجابة على الأسئلة بإجابات مختصرة ومباشرة.\n\n---\n\nFollow the following format.\n\nالسياق: من الممكن ان يحتوي على السياق\n\nالسؤال: {السؤال}\n\nالاستدلال: دعونا نفكر خطوة بخطوة من أجل {أنتج الإجابة}. نحن…\n\nالجواب: جواب مختصر\n\n---\n\nالسياق:\n[1] «وفقاً للبيانات الفعلية الصادرة عن الهيئة العامة للإحصاء؛ حقق الناتج المحلي الإجمالي للأنشطة غير النفطية للربع الأول من العام 2023م نمواً بمعدل 5.4%، وواصل أداءه الإيجابي خلال الربع الثاني بنمو بلغ 5.3%؛ مما يعكس نجاح جهود المملكة في تعزيز دور القطاع الخاص والتنوع الاقتصادي التي انعكست إيجاباً على أداء الأنشطة غير النفطية، إذ سجل نشاط الخدمات الجماعية والاجتماعية والشخصية ونشاط النقل والتخزين والاتصالات النمو الأعلى بمعدل 13.5% و9.9% على التوالي للربع الثاني 2023م، فيما سجل نشاط تجارة الجملة»\n[2] «النمو الأعلى بمعدل 13.5% و9.9% على التوالي للربع الثاني 2023م، فيما سجل نشاط تجارة الجملة والتجزئة والمطاعم والفنادق نمواً بمعدل 7.4%، يليه نشاط الصناعات التحويلية -باستثناء تكرير الزيت- بمعدل نمو 4.3%. كما سجل نشاط التشييد والبناء نموا

# dspy dataset preperation

In [15]:
formal_dataset = [dspy.Example(question=record["question_formal"], answer=str(record["answer"]), id=idx).with_inputs('question') for idx, record in df_formal.iterrows()]

In [16]:
from dspy.datasets import DataLoader

dl = DataLoader()

splits = dl.train_test_split(formal_dataset, train_size=0.33, random_state=42)
train_dataset = splits['train']
test_dataset = splits['test']

len(train_dataset), len(test_dataset)

(49, 101)

In [17]:
test_dataset[9]

Example({'question': 'ما هي كمية السلع الغذائية الأساسية التي تم توفيرها في عام 2023 للمساهمة في تحقيق الأمن الغذائي؟', 'answer': 'أكثر من 10 مليون طن.', 'id': 69}) (input_keys={'question'})

# Assess and Metric Definitions

## Assess
uses the same ollama model to assess the correctness of the answer generated by the dspy model against the gold answer

## Metric
usess the above assessment class to give a number [0, 1]

In [30]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of answer along the specified dimension."""

    assessed_text = dspy.InputField(desc="الجواب الذي يتم تقيمه:")
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="""
    جاوب باحد من الخيارات التالية:
    0: جواب غير صحيح
    0.5: الجواب "لا آعلم" آو "لا يوجد" او ما إلى ذلك ولكن يوجد جواب
    1: الجواب صحيح
    """)

In [31]:
def metric(gold, pred, trace=None):
    question, answer, gen_answer = gold.question, gold.answer, pred.answer

    correct = f"The text should answer `{question}` with `{answer}`. Does the assessed text contain this answer?"
    
    with dspy.context(lm=ollama_model):
        correct =  dspy.Predict(Assess)(assessed_text=gen_answer, assessment_question=correct)

    correct = float(correct.assessment_answer)
    score = correct if correct else 0

    return score

In [32]:
my_question = train_dataset[0]
response = uncompiled_rag(my_question.question)

metric(my_question, response)

1.0

In [33]:
from dspy.evaluate import Evaluate

evaluate_program = Evaluate(devset=test_dataset, metric=metric, num_threads=1, display_progress=True, display_table=50)

evaluate_program(uncompiled_rag)

Average Metric: 93.0 / 101  (92.1): 100%|████████████████████████████████████████████| 101/101 [04:12<00:00,  2.50s/it]


,question,example_answer,context,pred_answer,metric
0,ما هو عدد الميداليات التي حققتها المملكة في المسابقات الإقليمية والعالمية في عام 2023؟,60 ميدالية.,['ثانيًا: أهم المشاريع المنجزة للعام 2023م\nالسياحة\nحققت المملكة المرتبة الـ 13 عالميًا في مؤشر عدد السياح الدوليين لعام 2022م، والمرتبة الـ 11 عالميًا في مؤشر إيرادات...,حققوا ٦٠ ميدالية.,✔️ [1.0]
1,ما هو المستهدف لرؤية السعودية 2030 بشأن مشاركة المرأة في الشركات الاستشارية؟,nan,['35.3% متجاوزاً بذلك مستهدف رؤية السعودية 2030 عند مستوى 30%، مما يعكس جلياً ارتفاع نسبة الوعي بأهمية مشاركة المرأة ونجاح خطط ومبادرات تمكين المرأة التي...,معدل المشاركة المستهدفة لتمثيل المرأة في الشركات الاستشارية وفقاً لرؤية السعودية 2030 هي 30٪.,✔️ [1.0]
2,ما هو الهدف من إنشاء معمل البيانات في المركز الوطني لنظم الموارد الحكومية؟,دعم القطاع الحكومي عبر استكشاف البيانات وتحليلها ومواءمتها مع احتياجات المستفيدين وتعظيم الاستفادة منها باستخدام أحدث التقنيات لتحسين التخطيط وتعزيز كفاءة اتخاذ القرار.,['يُعنى المركز الوطني لنظم الموارد الحكومية بإرساء أسس الممارسات الرقمية الاحترافية في إدارة الموارد الحكومية والابتكار والتطوير وإدارة نظم الموارد المالية والبشرية والمشتريات الحكومية، حيث...,تحسين التخطيط واتخاذ القرار.,✔️ [1.0]
3,ما هو اسم المنصة الإلكترونية التي تم إطلاقها للحد من تعاملات الاقتصاد الخفي والتهرب الضريبي والتستر التجاري؟,فاتورة,['ثانيًا: أهم المشاريع المنجزة للعام 2023م\nالزكاة والضريبة والجمارك\nتنفيذ الفاتورة الإلكترونية (فاتورة) للحد من تعاملات الاقتصاد الخفي والتهرب الضريبي والتستر التجاري.\nإطلاق برنامج المشغل الاقتصادي الخليجي لتعزيز...,فاتورة,✔️ [1.0]
4,ما هي نسبة مساهمة القطاع الخاص غير النفطي في الناتج المحلي الإجمالي لعام 2022م؟,41%,['وفقاً للبيانات الفعلية الصادرة عن الهيئة العامة للإحصاء؛ حقق الناتج المحلي الإجمالي للأنشطة غير النفطية للربع الأول من العام 2023م نمواً بمعدل 5.4%، وواصل أداءه...,لا تتوفر معلومات دقيقة عن year المساهمة لعام 2022م تحديداً.,0.0
5,كم تنوي الحكومة الاقتراض من الصين؟,nan,['وتلبية للاحتياجات التمويلية، ووفقًا لخطة الاقتراض السنوية المعتمدة؛ من المتوقع الاستمرار في عمليات الاقتراض المحلية والخارجية لتمويل العجز المتوقع في الميزانية وسداد أصل الدين المستحق...,لا تشير الوثيقة إلى مبلغ محدد تقترضه الحكومة السعودية من الصين.,✔️ [1.0]
6,ما دور صندوق الاستثمارات العامة في تحقيق رؤية ٢٠٣٠؟,تطويــر ً مشـاريع كبـرى رائـدة محلي ً ـا وعالميـا، وإطـاق قطاعـات جديـدة تسـاهم فـي تحقيـق مسـتهدفات التنــوع االقتصــادي.,['يعد صندوق الاستثمارات العامة ركيزة أساسية في عملية التحول الاقتصادي وتحقيق مستهدفات رؤية السعودية 2030 عبر تنويع الاقتصاد، وتمكين القطاع الخاص، وتنمية القطاعات الواعدة، عبر...,صندوق الإستثمارات العامة هو أحد المحركات الأساسية للاقتصاد السعودي، ويساهم في تحقيق رؤية ٢٠٣٠ بتنويع الاقتصاد و ضخ الاستثمارات المحلية.,✔️ [1.0]
7,ما هو المستوى المحايد لمؤشر مديري المشتريات؟,٥٠ نقطة,['يعد مؤشر مديري المشتريات أحد أهم المؤشرات الاقتصادية التي تتابع حركة القطاع الخاص غير المنتج للنفط عالميًا، وقد أثرت حالة عدم الاستقرار للأسواق العالمية بشكل...,50 نقطة هي المستوى المحايد لمؤشر مديري المشتريات.,✔️ [1.0]
8,ما هو معدل النمو في الرقم القياسي لأسعار المستهلك منذ بداية العام الحالي وحتى شهر أكتوبر؟,2.50%,['ارتفع متوسط النمو في الرقم القياسي لأسعار المستهلك (معدل التضخم) منذ بداية العام الحالي حتى شهر أكتوبر بمعدل 2.5% مقارنة بالفترة نفسها من العام السابق،...,2.5%,✔️ [1.0]
9,ما هي كمية السلع الغذائية الأساسية التي تم توفيرها في عام 2023 للمساهمة في تحقيق الأمن الغذائي؟,أكثر من 10 مليون طن.,"['وفي إطار منجزات صندوق التنمية الزراعية فقد تم تمويل عدد 7,044 مستفيد بقيمة أكثر من 5.4 مليار ريال حتى سبتمبر من العام 2023م، كما تم...",تم تعزيز المخزون الاستراتيجي بـ 1.8 مليار ريال من السلع الغذائية الأساسية في عام 2023م.,✔️ [1.0]


92.08

## Problems in the metric

There are still some problems in the metric:
* if answer is 'nan' and model gives an answer it returns 1, see question 1, 14, etc
* sometimes incorrect answer is given 1, see 9, 11, etc
* sometimes model says idk and answer is idk but it returns 0.5, see 27

## Solutions

* Reduce the scope to write a simpler metric
* make a dspy program for the assessing model
    * before that maybe try CoT

In [51]:
from dspy.teleprompt import BootstrapFewShot

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=metric)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=train_dataset)

 10%|████████▍                                                                          | 5/49 [00:34<05:03,  6.89s/it]

Bootstrapped 4 full traces after 6 examples in round 0.


In [52]:
my_question = df_questions.iloc[20].question_formal
response = compiled_rag(my_question)
print(response.answer)

سجل نشاط الخدمات الجماعية والاجتماعية والشخصية أعلى معدل نمو بين الأنشطة غير النفطية للربع الثاني من العام بــــ 13.5%.


In [53]:
ollama_model.inspect_history(n=1)




الإجابة على الأسئلة بإجابات مختصرة ومباشرة.

---

السؤال: ما هو السبب الرئيسي لزيادة الإنفاق على باب المنافع السياسية لعام 2023م؟
الجواب: nan

السؤال: ما هو عدد الوظائف التي تم استحداثها في القطاع الثقافي حتى نهاية الربع الثالث من عام 2023م؟
الجواب: 1,947 وظيفة.

السؤال: ما هي نسبة العجز المتوقعة في الميزانية لعام 2023م كنسبة مئوية من الناتج المحلي الإجمالي القطري؟
الجواب: nan

السؤال: ما هو معدل نمو الودائع تحت الطلب حتى نهاية شهر سبتمبر من عام 2023م؟
الجواب: انخفاض بنسبة 4.8%

السؤال: لماذا تصدر الميزانية العامة؟
الجواب: يعـرض تفاصيـل الميزانيـة المعتمـدة علـى جانب الايرادات، والنفقات بحسـب التصنيف الاقتصـادي، وبحسـب القطاعـات وأهـم المشـاريع والبرامـج، كمـا يعـرض مسـتويات عجـز أو فائـض الميزانيـة والديـن العـام. كمـا يسـتعرض هـذا البيـان أبرز التطـورات المالية والاقتصاديــة لعــام 2023م، والاطــار المالــي والافــاق المســتقبلية لالقتصــاد علــى المــدى المتوســط، وأهــم الاســتراتيجيات والممكنــات الاقتصاديــة، وأهــم التحديــات التـي تواجـه الاقتصـاد خـال العـام المالـي القـادم

'\n\n\nالإجابة على الأسئلة بإجابات مختصرة ومباشرة.\n\n---\n\nالسؤال: ما هو السبب الرئيسي لزيادة الإنفاق على باب المنافع السياسية لعام 2023م؟\nالجواب: nan\n\nالسؤال: ما هو عدد الوظائف التي تم استحداثها في القطاع الثقافي حتى نهاية الربع الثالث من عام 2023م؟\nالجواب: 1,947 وظيفة.\n\nالسؤال: ما هي نسبة العجز المتوقعة في الميزانية لعام 2023م كنسبة مئوية من الناتج المحلي الإجمالي القطري؟\nالجواب: nan\n\nالسؤال: ما هو معدل نمو الودائع تحت الطلب حتى نهاية شهر سبتمبر من عام 2023م؟\nالجواب: انخفاض بنسبة 4.8%\n\nالسؤال: لماذا تصدر الميزانية العامة؟\nالجواب: يعـرض تفاصيـل الميزانيـة المعتمـدة علـى جانب الايرادات، والنفقات بحسـب التصنيف الاقتصـادي، وبحسـب القطاعـات وأهـم المشـاريع والبرامـج، كمـا يعـرض مسـتويات عجـز أو فائـض الميزانيـة والديـن العـام. كمـا يسـتعرض هـذا البيـان أبرز التطـورات المالية والاقتصاديــة لعــام 2023م، والاطــار المالــي والافــاق المســتقبلية لالقتصــاد علــى المــدى المتوســط، وأهــم الاســتراتيجيات والممكنــات الاقتصاديــة، وأهــم التحديــات التـي تواجـه الاقتصـاد خـال ا

In [54]:
evaluate_program_compiled = Evaluate(devset=test_dataset, metric=metric, num_threads=1, display_progress=True, display_table=101)

evaluate_program_compiled(compiled_rag)

Average Metric: 92.0 / 101  (91.1): 100%|████████████████████████████████████████████| 101/101 [09:45<00:00,  5.80s/it]


,question,example_answer,context,pred_answer,metric
0,ما هو عدد الميداليات التي حققتها المملكة في المسابقات الإقليمية والعالمية في عام 2023؟,60 ميدالية.,['ثانيًا: أهم المشاريع المنجزة للعام 2023م\nالسياحة\nحققت المملكة المرتبة الـ 13 عالميًا في مؤشر عدد السياح الدوليين لعام 2022م، والمرتبة الـ 11 عالميًا في مؤشر إيرادات...,حققت المملكة العربية السعودية ما مجموعه ستون (60) ميدالية في مسابقات إقليمية وعالمية مختلفة خلال العام 2023.,✔️ [1.0]
1,ما هو المستهدف لرؤية السعودية 2030 بشأن مشاركة المرأة في الشركات الاستشارية؟,nan,['35.3% متجاوزاً بذلك مستهدف رؤية السعودية 2030 عند مستوى 30%، مما يعكس جلياً ارتفاع نسبة الوعي بأهمية مشاركة المرأة ونجاح خطط ومبادرات تمكين المرأة التي...,تهدف رؤية السعودية 2030 إلى تحقيق نسبة مشاركة للمرأة في سوق العمل تبلغ 30%.,✔️ [1.0]
2,ما هو الهدف من إنشاء معمل البيانات في المركز الوطني لنظم الموارد الحكومية؟,دعم القطاع الحكومي عبر استكشاف البيانات وتحليلها ومواءمتها مع احتياجات المستفيدين وتعظيم الاستفادة منها باستخدام أحدث التقنيات لتحسين التخطيط وتعزيز كفاءة اتخاذ القرار.,['يُعنى المركز الوطني لنظم الموارد الحكومية بإرساء أسس الممارسات الرقمية الاحترافية في إدارة الموارد الحكومية والابتكار والتطوير وإدارة نظم الموارد المالية والبشرية والمشتريات الحكومية، حيث...,يهدف معمل البيانات الذي أنشئ في المركز الوطني لنظم الموارد الحكومية إلى دعم القطاع الحكومي من خلال استكشاف وتحليل البيانات وتكييفها وفقًا لاحتياجات المستفيدين وتعظيم...,✔️ [1.0]
3,ما هو اسم المنصة الإلكترونية التي تم إطلاقها للحد من تعاملات الاقتصاد الخفي والتهرب الضريبي والتستر التجاري؟,فاتورة,['ثانيًا: أهم المشاريع المنجزة للعام 2023م\nالزكاة والضريبة والجمارك\nتنفيذ الفاتورة الإلكترونية (فاتورة) للحد من تعاملات الاقتصاد الخفي والتهرب الضريبي والتستر التجاري.\nإطلاق برنامج المشغل الاقتصادي الخليجي لتعزيز...,الفاتورة الإلكترونية (فاتورة) هي المنصة التي أطلقت لخدمة القطاع المالي والضريبي.,✔️ [1.0]
4,ما هي نسبة مساهمة القطاع الخاص غير النفطي في الناتج المحلي الإجمالي لعام 2022م؟,41%,['وفقاً للبيانات الفعلية الصادرة عن الهيئة العامة للإحصاء؛ حقق الناتج المحلي الإجمالي للأنشطة غير النفطية للربع الأول من العام 2023م نمواً بمعدل 5.4%، وواصل أداءه...,ساهم القطاع الخاص غير النفطي بنسبة 68٪ في الناتج المحلي الإجمالي لعام 2022.,0.0
5,كم تنوي الحكومة الاقتراض من الصين؟,nan,['وتلبية للاحتياجات التمويلية، ووفقًا لخطة الاقتراض السنوية المعتمدة؛ من المتوقع الاستمرار في عمليات الاقتراض المحلية والخارجية لتمويل العجز المتوقع في الميزانية وسداد أصل الدين المستحق...,لا يشير النص إلى مبلغ محدد تعتزم الحكومة السعودية الاقتراضه من الصين، حيث تركز استراتيجيتها على استكشاف أسواق تمويل ومنهجيات جديدة وتنويع مصادر التمويل. تبدو...,✔️ [1.0]
6,ما دور صندوق الاستثمارات العامة في تحقيق رؤية ٢٠٣٠؟,تطويــر ً مشـاريع كبـرى رائـدة محلي ً ـا وعالميـا، وإطـاق قطاعـات جديـدة تسـاهم فـي تحقيـق مسـتهدفات التنــوع االقتصــادي.,['يعد صندوق الاستثمارات العامة ركيزة أساسية في عملية التحول الاقتصادي وتحقيق مستهدفات رؤية السعودية 2030 عبر تنويع الاقتصاد، وتمكين القطاع الخاص، وتنمية القطاعات الواعدة، عبر...,يلعب صندوق الإستثمارت العامة دوراً أساسياً فى دفع عجلة الاقتصاد والإستثمار بالمملكة من خلال تطوير مشاريع رائدة محلية وعالمية وإطلاق قطاعات جديدة تساهم في تحقيق...,✔️ [1.0]
7,ما هو المستوى المحايد لمؤشر مديري المشتريات؟,٥٠ نقطة,['يعد مؤشر مديري المشتريات أحد أهم المؤشرات الاقتصادية التي تتابع حركة القطاع الخاص غير المنتج للنفط عالميًا، وقد أثرت حالة عدم الاستقرار للأسواق العالمية بشكل...,المستوى المحايد لمؤشر مديري المشتريات هو ٥٠ نقطة.,✔️ [1.0]
8,ما هو معدل النمو في الرقم القياسي لأسعار المستهلك منذ بداية العام الحالي وحتى شهر أكتوبر؟,2.50%,['ارتفع متوسط النمو في الرقم القياسي لأسعار المستهلك (معدل التضخم) منذ بداية العام الحالي حتى شهر أكتوبر بمعدل 2.5% مقارنة بالفترة نفسها من العام السابق،...,ارتفع متوسط الرقم القياسي لأسعار المستهلك بنسبة 2.5٪ منذ بداية العام وحتى أكتوبر.,✔️ [1.0]
9,ما هي كمية السلع الغذائية الأساسية التي تم توفيرها في عام 2023 للمساهمة في تحقيق الأمن الغذائي؟,أكثر من 10 مليون طن.,"['وفي إطار منجزات صندوق التنمية الزراعية فقد تم تمويل عدد 7,044 مستفيد بقيمة أكثر من 5.4 مليار ريال حتى سبتمبر من العام 2023م، كما تم...",ساهم صندوق التنمية الزراعية بتوفير أكثر من 1.8 م

91.09